## Métricas para el sistema de recomendación

In [2]:
import pandas as pd
import numpy as np
from data_reader import read_data
from recommender import Recommender

def cargar_y_preparar_datos():
    """
    Carga los datos necesarios para el sistema de recomendación y prepara el conjunto de datos de ratings.

    Retorna:
        tuple: Contiene DataFrames de géneros de películas, ratings y metadatos de películas.
    """
    md_genres, ratings, md = read_data()
    ratings = ratings.loc[ratings['rating'] != 0]
    return md_genres, ratings, md

def obtener_datos_de_prueba(ratings):
    """
    Selecciona un subconjunto de datos como conjunto de prueba.

    Parámetros:
        ratings (DataFrame): DataFrame de ratings de películas.

    Retorna:
        DataFrame: Conjunto de datos de prueba.
    """
    return ratings.sample(frac=0.3, random_state=42)

def generar_recomendaciones(test_data, md_genres, ratings, md):
    """
    Utiliza el modelo de recomendador para generar recomendaciones de películas.

    Parámetros:
        test_data (DataFrame): Datos de prueba para generar recomendaciones.
        md_genres (DataFrame): DataFrame de géneros de películas.
        ratings (DataFrame): DataFrame de ratings de películas.
        md (DataFrame): DataFrame de metadatos de películas.

    Retorna:
        list: Lista de recomendaciones con cada elemento conteniendo (userId, movieId, rating).
    """
    rec = Recommender(1)
    predictions = rec.recommend_movies_for_test(test_data, md_genres, ratings, md)
    return [item for sublist in predictions for item in sublist]

def crear_dataframe_predicciones(pred):
    """
    Crea un DataFrame a partir de las predicciones generadas.

    Parámetros:
        pred (list): Lista de tuplas con userId, movieId, y rating.

    Retorna:
        DataFrame: DataFrame de predicciones filtrado por un rating mínimo.
    """
    df_predictions = pd.DataFrame(pred, columns=['userId', 'movieId', 'rating'])
    return df_predictions.loc[df_predictions['rating'] > 3],df_predictions  # Filtra por ratings mayores a 3


md_genres, ratings, md = cargar_y_preparar_datos()
test_data = obtener_datos_de_prueba(ratings)
pred = generar_recomendaciones(test_data, md_genres, ratings, md)
df_recommendation,df_predictions = crear_dataframe_predicciones(pred)

In [3]:
import pandas as pd
import numpy as np

def evaluar_recomendaciones(df_recommendation, test_data):
    """
    Evalúa las recomendaciones comparando contra los datos de prueba y calcula métricas como precisión, recall y F1.

    Parámetros:
        df_recommendation (DataFrame): DataFrame que contiene las recomendaciones hechas a los usuarios.
        test_data (DataFrame): DataFrame que contiene los datos de prueba reales para comparar con las recomendaciones.

    Retorna:
        tuple: Tupla que contiene listas de precisión, recall, F1 y conteos para cálculos micro.
    """
    # Fusionar los datos recomendados con los datos de prueba para obtener intersecciones
    test_data_recommended = pd.merge(df_recommendation[['userId', 'movieId']], test_data, on=['userId', 'movieId'], how='inner')
    # Filtrar por aquellos que tienen calificación mayor a 3 en los datos de prueba
    valid = test_data_recommended.loc[test_data_recommended['rating'] > 3]

    # Inicializar listas para almacenar las métricas por usuario y para cálculos micro
    precisions = []
    recalls = []
    f1_scores = []
    hits_per_user = []
    recset_per_user = []
    testset_per_user = []

    # Iterar sobre cada usuario único en los datos validados
    for user in valid['userId'].unique():
        hits = valid.loc[valid['userId'] == user].shape[0]
        recset = df_recommendation.loc[df_recommendation['userId'] == user].shape[0]
        testset = test_data.loc[test_data['userId'] == user].shape[0]

        # Calcular Precision, Recall y F1 por usuario
        Precision = hits / recset if recset > 0 else 0
        Recall = hits / testset if testset > 0 else 0
        F1 = 2 * (Precision * Recall) / (Precision + Recall) if (Precision + Recall) > 0 else 0

        # Almacenar las métricas calculadas
        precisions.append(Precision)
        recalls.append(Recall)
        f1_scores.append(F1)

        # Almacenar los valores para el cálculo micro
        hits_per_user.append(hits)
        recset_per_user.append(recset)
        testset_per_user.append(testset)


        print(f'Precision: {Precision}')
        print(f'Recall: {Recall}')
        print(f'F1: {F1}')

    return precisions, recalls, f1_scores, hits_per_user, recset_per_user, testset_per_user, valid


precisions, recalls, f1_scores, hits_per_user, recset_per_user, testset_per_user, valid = evaluar_recomendaciones(df_recommendation, test_data)


Precision: 0.7058823529411765
Recall: 0.45569620253164556
F1: 0.553846153846154
Precision: 0.538860103626943
Recall: 0.30057803468208094
F1: 0.38589981447124305
Precision: 0.29012345679012347
Recall: 0.13073713490959665
F1: 0.18024928092042186
Precision: 0.7638888888888888
Recall: 0.44
F1: 0.5583756345177665
Precision: 0.5333333333333333
Recall: 0.38095238095238093
F1: 0.4444444444444444
Precision: 0.8214285714285714
Recall: 0.45098039215686275
F1: 0.5822784810126583
Precision: 0.8448275862068966
Recall: 0.6712328767123288
F1: 0.7480916030534351
Precision: 0.7142857142857143
Recall: 0.45454545454545453
F1: 0.5555555555555556
Precision: 0.782608695652174
Recall: 0.38095238095238093
F1: 0.5124555160142349
Precision: 0.8536585365853658
Recall: 0.5303030303030303
F1: 0.6542056074766355
Precision: 0.13333333333333333
Recall: 0.047244094488188976
F1: 0.06976744186046512
Precision: 0.7159090909090909
Recall: 0.5294117647058824
F1: 0.6086956521739131
Precision: 0.6666666666666666
Recall: 0.338

In [4]:
def average_precision(predicted_ratings, isHitFunc, getPropertyFunc):
    """
    Calcula la Precisión Promedio (AP) para las recomendaciones de un usuario.

    Parámetros:
        predicted_ratings (iterable): Calificaciones predichas o recomendaciones.
        isHitFunc (function): Función que determina si una recomendación es un acierto.
        getPropertyFunc (function): Función que extrae la propiedad relevante de una recomendación.

    Retorna:
        float: El valor de Average Precision para el conjunto de recomendaciones.
    """
    rel = 0
    numerator = 0
    for index, rating in enumerate(predicted_ratings):
        if isHitFunc(getPropertyFunc(rating)):
            rel += 1
            numerator += (rel / (index + 1))
    return numerator / rel if rel > 0 else 0

def mean_average_precision(df_recommendation, test_data, valid):
    """
    Calcula el Mean Average Precision (MAP) para todas las recomendaciones.

    Parámetros:
        df_recommendation (DataFrame): DataFrame con las recomendaciones.
        test_data (DataFrame): DataFrame con los datos de prueba.
        valid (DataFrame): DataFrame con las interacciones validadas (calificaciones > 3).

    Retorna:
        float: El valor de MAP calculado.
    """
    average_precisions = []
    for user in df_recommendation['userId'].unique():
        user_recommendations = df_recommendation[df_recommendation['userId'] == user]
        user_valid = valid[valid['userId'] == user]

        ap = average_precision(
            user_recommendations.itertuples(),
            lambda movieId: movieId in user_valid['movieId'].values,
            lambda row: row.movieId
        )
        average_precisions.append(ap)
    return np.mean(average_precisions) if average_precisions else 0

def calcular_metricas_globales(precisions, recalls, f1_scores, hits_per_user, recset_per_user, testset_per_user):
    """
    Calcula y muestra las métricas macro y micro para todas las recomendaciones.

    Parámetros:
        precisions (list): Lista de precisiones calculadas para cada usuario.
        recalls (list): Lista de recalls calculados para cada usuario.
        f1_scores (list): Lista de F1 scores calculados para cada usuario.
        hits_per_user (list): Lista de aciertos por usuario.
        recset_per_user (list): Lista del tamaño del conjunto de recomendaciones por usuario.
        testset_per_user (list): Lista del tamaño del conjunto de pruebas por usuario.
    """
    # Calcular métricas macro
    avg_precision = np.mean(precisions)
    avg_recall = np.mean(recalls)
    avg_f1 = np.mean(f1_scores)

    # Calcular métricas micro
    total_hits = sum(hits_per_user)
    total_recset = sum(recset_per_user)
    total_testset = sum(testset_per_user)

    micro_precision = total_hits / total_recset if total_recset > 0 else 0
    micro_recall = total_hits / total_testset if total_testset > 0 else 0
    micro_f1 = 2 * (micro_precision * micro_recall) / (micro_precision + micro_recall) if (micro_precision + micro_recall) > 0 else 0

    print(f'Average Precision (Macro): {avg_precision:.4f}')
    print(f'Average Recall (Macro): {avg_recall:.4f}')
    print(f'Average F1 (Macro): {avg_f1:.4f}')
    print(f'Precision (Micro): {micro_precision:.4f}')
    print(f'Recall (Micro): {micro_recall:.4f}')
    print(f'F1 (Micro): {micro_f1:.4f}')


calcular_metricas_globales(precisions, recalls, f1_scores, hits_per_user, recset_per_user, testset_per_user)
map_score = mean_average_precision(df_recommendation, test_data, valid)
print(f'MAP: {map_score:.4f}')


Average Precision (Macro): 0.7310
Average Recall (Macro): 0.4725
Average F1 (Macro): 0.5625
Precision (Micro): 0.7020
Recall (Micro): 0.4077
F1 (Micro): 0.5158
MAP: 0.7614
